<h1><center>Story of what happens in American's life</center></h1>

It is a well-known fact that a complete rotation of the planet Earth takes 24 hours causing the formation of day and night. The fascinating part is to analyze how the people in the world decide to spend the available precious 24 hours. The demographics plays a very vital role on analysing and understanding the life-style of people mainly due to the influence of the time-zone, culture, population, socio-economy factors, current trend, etc. In this article, we focus on visually studying the behaviour of person living in the United States for the most obvious reasons that the country comprises of diverse, multi-cultural and a democratic-republic population.

To go another step ahead, this article shows how an American spends his/her day which is classified into a number of activities like Personal Care, Travelling, Socialising-Leisure time, Religion, Work, Sports- Fitness and so on.

In [1]:
#nbi:hide_in
import pandas as pd
import bqplot as bq
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import ipywidgets

In [3]:
#nbi:hide_in
cps = pd.read_csv('https://raw.githubusercontent.com/sarvaniputta/nbinteract_tutorial/master/atuscps_2017.csv')
summary = pd.read_csv('https://raw.githubusercontent.com/sarvaniputta/nbinteract_tutorial/master/atussum_2017.csv') #Summary file of 2017
cps_ = cps.groupby('TUCASEID').GESTFIPS.first().to_frame()

merged = summary.merge(cps_, left_on = 'TUCASEID', right_index = True)

leisure_columns = [x for x in merged.columns if x.startswith('t1203')]
work_columns = [x for x in merged.columns if x.startswith('t0501')]
leisure_by_state = merged.loc[:, leisure_columns].groupby(merged.GESTFIPS).mean()

work_by_state = merged.loc[:, work_columns].groupby(merged.GESTFIPS).mean()


time_spent = merged.loc[:, ['TUCASEID']+leisure_columns].set_index('TUCASEID').sum(axis=1)


def normalized_hist(data, bins=10):
    counts, bins = np.histogram(data, bins=bins)
    counts = counts*100/counts.sum()
    return bins, counts   


cscale = bq.ColorScale(scheme = 'viridis')        
                   # reverse the colorscale or Hawaii is not visible

map_tt = bq.Tooltip(labels = ['State', 'Leisure time (minutes)'], fields = ['name', 'color'])
sc_geo = bq.AlbersUSA()
states_map = bq.Map(color = leisure_by_state.sum(axis=1).round(1).to_dict(), 
                     map_data=bq.topo_load('map_data/USStatesMap.json'),
                    scales = {'projection':sc_geo, 'color':cscale},tooltip = map_tt,
                    interactions = {'click': 'select', 'hover':'tooltip', },
                    anchor_style = {'fill':'red'}, 
                    selected_style = {'opacity': 1.0},
                    unselected_style = {'opacity': 1.0})

cax = bq.ColorAxis(scale=cscale, orientation='vertical', side='left')
fig_map = bq.Figure(marks=[states_map],axes=[cax], title = 'Average time spent for leisure')
fig_map.fig_margin = {'bottom': 10, 'top': 10, 'left': 0, 'right': 0};

               
sc_x2 = bq.LinearScale()
sc_y2 = bq.LinearScale()
ax_x2 = bq.Axis(label = 'Time spent for leisure(minutes)', scale=sc_x2, orientation='horizontal')
ax_y2 = bq.Axis(label='Frequency%', scale=sc_y2,orientation='vertical')
x, y = normalized_hist(time_spent, bins=14)
bars = bq.Bars(x = x[:-1], y=y, align='right', scales={'x': sc_x2, 'y': sc_y2})
fig_bar = bq.Figure(marks = [bars],axes=[ax_x2, ax_y2]);


def on_select(change):
    
    if not change['new']:
        selected_fips = merged.GESTFIPS.unique()
    else:
        selected_fips = change['new']
        states_map.selected = [selected_fips[-1]]
    activity = activity_dropdown.value #Leisure time
    if activity == 'Leisure Time':
        columns = [x for x in merged.columns if x.startswith('t1203')]
    elif activity == 'Work Time':
        columns = [x for x in merged.columns if x.startswith('t0501')]
    x, y = normalized_hist(merged
                            .loc[merged.GESTFIPS.isin(selected_fips), ['TUCASEID']+columns]
                            .set_index('TUCASEID')
                            .sum(axis=1), bins=14)
    bars.x = x
    bars.y = y
states_map.observe(on_select, 'selected')

activity_dropdown = ipywidgets.Dropdown(options = ['Leisure Time','Work Time'],
                    value='Leisure Time', Description = 'Select activity')



def on_activity_change(change):
    states_map.selected=[]
    activity = activity_dropdown.value #Leisure time
    if activity == 'Leisure Time':
        columns = [x for x in merged.columns if x.startswith('t1203')]
        ax_x2.label = 'Time spent for leisure(minutes)'
        fig_map.title = 'Average time spent for leisure'
        map_tt.labels = ['State', 'Leisure time (minutes)']
    elif activity == 'Work Time':
        columns = [x for x in merged.columns if x.startswith('t0501')]
        ax_x2.label = 'Time spent working(minutes)'
        fig_map.title = 'Average time spent working'
        map_tt.labels = ['State', 'Working time (minutes)']
    filtered = merged.loc[:, columns].groupby(merged.GESTFIPS).mean().sum(axis=1).round(1).to_dict()
    states_map.color = filtered
    

activity_dropdown.observe(on_activity_change, 'value')
ipywidgets.VBox([activity_dropdown,ipywidgets.HBox([fig_map, fig_bar])])